In [1]:
import pandas as pd
import datetime as dt
import os

os.chdir("./data/stocks_data")

In [2]:
adj_close = pd.read_parquet("adj_close.parquet")
return_tommorrow = pd.read_parquet("return_tomorrow.parquet")

trade_dates = adj_close.index.to_list()[:-1]

adj_close.head()

,A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACGL,ACN,ACS,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1996-01-02,NaN,NaN,NaN,0.242501,NaN,7.250,4.985257,2.569444,NaN,NaN,...,7.238386,8.375014,5.400020,31.252378,NaN,NaN,NaN,15.388889,11.960916,NaN
1996-01-03,NaN,NaN,NaN,0.242501,NaN,7.375,5.000181,2.541667,NaN,NaN,...,7.329781,8.400988,5.468374,31.223751,NaN,NaN,NaN,15.666667,11.696796,NaN
1996-01-04,NaN,NaN,NaN,0.238255,NaN,7.500,5.074811,2.555556,NaN,NaN,...,7.238386,8.452925,5.451284,30.565504,NaN,NaN,NaN,14.000000,11.206285,NaN
1996-01-05,NaN,NaN,NaN,0.258543,NaN,7.500,4.970332,2.500000,NaN,NaN,...,7.293225,8.660674,5.365843,30.222082,NaN,NaN,NaN,13.444444,11.244011,NaN
1996-01-08,NaN,NaN,NaN,0.261373,NaN,7.125,5.059889,2.500000,NaN,NaN,...,7.256669,8.764545,5.331663,30.164845,NaN,NaN,NaN,13.444444,11.130821,NaN


In [3]:
# generate all date_ranges
print(len(trade_dates))

# drop some dates at the begining to make the number of dates dividable by 250
n_drop = len(trade_dates) % 250
if n_drop != 0:
    trade_dates = trade_dates[n_drop:]

print(len(trade_dates))

7129
7000


In [4]:
# generate date_ranges, each with length 1000 and step 250
ranges = []
for start in range(0, len(trade_dates) - 1000 + 1, 250):
    end = start + 1000
    ranges.append(trade_dates[start:end])
print(len(ranges))

25


In [6]:
def generate_single_period(dates_range):
    period_close = adj_close.loc[dates_range]
    period_close = period_close.drop(columns=period_close.columns[period_close.count() < 600])
    period_close = period_close.stack().reset_index()
    period_close.columns = ["Date", "Ticker", "Adj_Close"]

    period_rtn = return_tommorrow.loc[dates_range].stack().reset_index()
    period_rtn.columns = ["Date", "Ticker", "Return_tomorrow"]
    
    period_data = pd.merge(period_close, period_rtn, on=["Date", "Ticker"], how="left")
    period_data = period_data.dropna(how="any", axis=0)
    if period_data.Date.nunique() != len(dates_range):
        print("Error: date range not match")
    return period_data.sort_values(by=["Ticker", "Date"]).reset_index(drop=True)

In [7]:
from tqdm import tqdm

save_path = "./single_period_data"
os.makedirs(save_path, exist_ok=True)

for i, period_dates in tqdm(enumerate(ranges), total=len(ranges)):
    data = generate_single_period(period_dates)
    data.to_parquet(os.path.join(save_path, f"period_{i}.parquet"))

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:16<00:00,  1.49it/s]
